In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [28]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
DOLLAR_THRESHOLD = "115009542m"

START_DATE = "20240101"
END_DATE = "20241231"

# Build base name
if SAMPLE_TYPE == "dollar":
    RESAMPLED_NAME = f"{SYMBOL}-{DOLLAR_THRESHOLD}-dollar-{START_DATE}-{END_DATE}"
else:
    RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"

In [29]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)

CPU times: user 2.14 ms, sys: 46.7 ms, total: 48.9 ms
Wall time: 56.2 ms


In [30]:
df.shape

(371236, 100)

In [31]:
df.head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 01:44:00,1.10381,1.103845,1.103805,1.103835,2.090600e+11,0.000037,0.000020,0.000018,0.000018,0.000016,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:45:00,1.10385,1.104010,1.103850,1.103890,3.001600e+11,0.000033,0.000055,0.000050,0.000050,0.000026,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:46:00,1.10390,1.103960,1.103885,1.103905,3.997200e+11,0.000034,0.000015,0.000014,0.000014,0.000026,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:47:00,1.10391,1.103990,1.103860,1.103900,5.193100e+11,0.000032,-0.000005,-0.000005,-0.000005,0.000024,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:48:00,1.10389,1.103955,1.103870,1.103900,1.983200e+11,0.000028,0.000000,0.000000,0.000000,0.000015,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025


In [32]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_delta',
       'close_return', 'close_log_return', 'ret_mean_5', 'ret_mean_10',
       'ret_mean_15', 'ret_mean_20', 'log_volume', 'rv5', 'log_rv5',
       'sqrt_rv5', 'rv15', 'log_rv15', 'sqrt_rv15', 'rv50', 'log_rv50',
       'sqrt_rv50', 'ema5', 'ema5_slope', 'close_above_ema5', 'ema20',
       'ema20_slope', 'close_above_ema20', 'ema50', 'ema50_slope',
       'close_above_ema50', 'ema100', 'ema100_slope', 'close_above_ema100',
       'atr14', 'atr60', 'atr120', 'log_atr14', 'atr14_percent',
       'atr14_adjusted_return', 'log_atr60', 'atr60_percent',
       'atr60_adjusted_return', 'log_atr120', 'atr120_percent',
       'atr120_adjusted_return', 'adx14', 'plus_di14', 'minus_di14',
       'bb_upper', 'bb_lower', 'bb_mavg', 'bb_width', 'bb_position',
       'dc20_upper', 'dc20_lower', 'dc20_mid', 'dc20_width',
       'close_above_dc20_mid', 'dc20_breakout', 'dc20_breakdown', 'dc50_upper',
       'dc50_lower', 'dc50_mi

## Calculate Daily Volatility

In [33]:
vol = df['close_log_return'].rolling(window=60).std() # 計算 60 分鐘的 標準差
vol.mean()

np.float64(8.42508166690778e-05)

## Get Events Start Time (t0)

In [34]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def get_tevents_optimized(data: pd.Series, threshold: float) -> pd.DatetimeIndex:
    # get tEvents using CUSUM filter
    values = data.values
    timestamps = data.index

    s_pos = np.zeros_like(values)
    s_neg = np.zeros_like(values)

    t_events_mask = np.zeros_like(values, dtype=bool)

    cum_pos, cum_neg = 0.0, 0.0

    for i in tqdm(range(len(values))):
        cum_pos = max(0.0, cum_pos + values[i])
        cum_neg = min(0.0, cum_neg + values[i])
        s_pos[i] = cum_pos
        s_neg[i] = cum_neg

        if cum_pos > threshold:
            t_events_mask[i] = True
            cum_pos = 0.0
        if cum_neg < -threshold:
            t_events_mask[i] = True
            cum_neg = 0.0

    return timestamps[t_events_mask]


In [35]:
EVENT_NAME = 'CUSUM'
THRESHOLD = vol.mean()*2
EVENT_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}_{THRESHOLD:.2e}.pkl"

In [36]:
%%time

t_events = get_tevents_optimized(df['close_log_return'].iloc[1:], threshold=THRESHOLD)

100%|█████████████████████████████████| 371235/371235 [00:00<00:00, 3120404.42it/s]

CPU times: user 119 ms, sys: 3.48 ms, total: 123 ms
Wall time: 124 ms


In [37]:
t_events

DatetimeIndex(['2024-01-02 01:56:00', '2024-01-02 02:01:00',
               '2024-01-02 02:07:00', '2024-01-02 02:15:00',
               '2024-01-02 02:23:00', '2024-01-02 02:33:00',
               '2024-01-02 02:41:00', '2024-01-02 02:49:00',
               '2024-01-02 02:57:00', '2024-01-02 03:07:00',
               ...
               '2024-12-30 22:00:00', '2024-12-30 22:04:00',
               '2024-12-30 22:05:00', '2024-12-30 22:33:00',
               '2024-12-30 22:52:00', '2024-12-30 23:00:00',
               '2024-12-30 23:05:00', '2024-12-30 23:14:00',
               '2024-12-30 23:27:00', '2024-12-30 23:34:00'],
              dtype='datetime64[ns]', name='timestamp', length=64366, freq=None)

In [38]:
EVENT_PATH

PosixPath('../data/events/USDJPY-1m-20240101-20241231_CUSUM_1.69e-04.pkl')

In [39]:
t_events.to_series().to_pickle(EVENT_PATH)